Tilføj forklarende tekst a la 02_heterologous patway (...)

Undersøg enhederne.

Overvej (ikke nødvendigt): 
 - Prøv at ændre mediet på andre måder 
        - E.g. udskift glucose
        - tilføj precursor i mediet.



In [1]:
# Load packages needed
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

In [47]:
model = read_sbml_model('models/iMM904_progesterone.xml') # Load model

In [48]:
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand") # Because it is a reversible reaction, we need to set a demand for production of progesterone 
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}') 

Adding reaction progesterone demand: progesterone_c --> 


# Maximum theoretical productivity and yield of progesterone on default medium 

In order to asses the model, it can be beneficial to look at the maximum theoretical growth rate, as well as the productivity and yield obtained from the compound of interest. For this, we must first find the nutrients of the growth medium and the quantity of each nutrient.

In [4]:
# See medium compositions
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

It can bee seen that the two limiting exchanges are glucose and oxygen, and that glucose is the sole carbon source of the default growth medium. 

In [52]:
model.objective.expression

1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090

In [68]:
with model:
    baseline_max_growth = model.optimize().objective_value
    model.objective = model.reactions.R02216
    baseline_max_production = model.optimize().objective_value
print(f"Maximum growth: {baseline_max_growth}")
print(f"Maximum progesterone production: {baseline_max_production}")

Maximum growth: 0.28786570370401676
Maximum progesterone production: 0.14285714285714285


In [69]:
with model: 
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    medium = model.medium
    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth} # objective is to lead flux through the reaction producing progesterone top maximise production
    solution = model.optimize()
    progesterone_production = solution.fluxes['DM_progesterone_c']
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose concentration: ',format(medium[Glucose_exchange.id]))
    print('Maximum theoretical biomass productivity: ', round(solution.fluxes['BIOMASS_SC5_notrace'],5), '/h')
    print('Maximum theoretical productivity of progesterone on glucose: ', round(progesterone_production,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on glucose: ', round(max_yield,5), 'mmol progesterone/mmol glucose') # calculate yield on galactose

Glucose concentration:  10.0
Maximum theoretical biomass productivity:  0.13131 /h
Maximum theoretical productivity of progesterone on glucose:  0.13475 mmol/gDW*h
Maximum theoretical yield of progesterone on glucose:  0.01347 mmol progesterone/mmol glucose
0.142857142857143*BIOMASS_SC5_notrace - 0.142857142857143*BIOMASS_SC5_notrace_reverse_93090 + 0.287865703704017*DM_progesterone_c - 0.287865703704017*DM_progesterone_c_reverse_f69dc


In [71]:
model.objective.expression

1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090

The productivity is 0.14286 mmol/gDW*h and the yield of progesterone is 0.01429 progesterone/mmol glucose. Now it would be interesting to see what happens to the production of progesterone, if we make more glucose available for uptake in the growth medium. Note that when the objective is set to the production of one compound, the biomass becomes zero as it is not viable for the cell.

## Maximum theoretical productivity and yield in Cmole

To account for cabon molecules, instead of mass, we can convert the productivity and yield to Cmole.

In [6]:
# Define conversion factors
Cmole_glucose = 6
Cmole_progesterone = 21

In [7]:
progesterone_production_Cmole = (progesterone_production)/(Cmole_progesterone) # Max productivity in Cmole
max_yield_Cmole = (max_yield*Cmole_progesterone)/(Cmole_glucose) # Max yield in Cmole
print('Maximum theoretical productivity of progesterone:', round(progesterone_production_Cmole,5), 'Cmole/gDW*h') # Print max productivity in Cmole
print('Maximum theoretical yield of progesterone:', round(max_yield_Cmole,5), 'Cmole progesterone/Cmole glucose') # Print max yield on glucose in Cmole

Maximum theoretical productivity of progesterone: 0.0068 Cmole/gDW*h
Maximum theoretical yield of progesterone: 0.05 Cmole progesterone/Cmole glucose


# Increasing glucose availability in the growth medium

Ass seen before, one of the rate limiting compounds is glucose. As glucose is only available in a low quantity in the medium, it could be interesting to increase the concentration of glucose in the medium and see how it affects the theoretical growth rate of the strain.

In [9]:
# Simulate the model at different glucose concentrations with biomass as objective
with model:
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') # Define D-glucose exchange reaction
    medium = model.medium 
    objective_value = model.reactions.BIOMASS_SC5_notrace # Objective is to maximise growth
    growth = model.optimize().objective_value # Simulate model 
    print('At glucose concentration:', format(medium[Glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h') # since the objective is biomass, not flux, the unit changes to /h 
    print('')

    medium[Glucose_exchange.id] = 500 # Set D-glucose to half-max (500)
    model.medium = medium
    growth = model.optimize().objective_value 
    print('At glucose concentration:', format(medium[Glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h')
    print('')

    medium[Glucose_exchange.id] = 1000 # Set D-glucose to max (1000)
    model.medium = medium
    growth = model.optimize().objective_value 
    print('At glucose concentration:', format(medium[Glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h')



At glucose concentration: 10.0
Maximum theoretical growth rate: 0.28787 /h

At glucose concentration: 500
Maximum theoretical growth rate: 11.06243 /h

At glucose concentration: 1000
Maximum theoretical growth rate: 21.89736 /h


It can be seen that the growth rate is increased when the availability of glucose in the growth medium is increased. Hence, it could be interesting to simulate the effect of increasing glucose in the medium on the production of progesterone.

## The effect on progesterone production by increasing glucose availability

Now, we will simulate the effect that increasing the glucose concentration in the medium has on the production of progesterone.

In [10]:
# Simulate the model with increasing glucose concentrations with progesterone production as objective
with model: 
    medium = model.medium
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') # Define D-glucose exchange reaction
    medium = model.medium 
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution = model.optimize() # simulate model
    progesterone_production = solution.objective_value
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose concentration: ',format(medium[Glucose_exchange.id]))
    print('Maximum theoretical biomass productivity: ', round(solution.fluxes['BIOMASS_SC5_notrace'],5), '/h')
    print('Maximum theoretical productivity of progesterone on glucose: ', round(progesterone_production,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on glucose: ', round(max_yield,5), 'mmol progesterone/mmol glucose') # calculate yield on galactose
    print('')

    medium[Glucose_exchange.id] = 500 # Set D-glucose to half-max
    model.medium = medium 

    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution = model.optimize() # simulate model
    progesterone_production = solution.objective_value
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose concentration: ',format(medium[Glucose_exchange.id]))
    print('Maximum theoretical biomass productivity: ', round(solution.fluxes['BIOMASS_SC5_notrace'],5), '/h')
    print('Maximum theoretical productivity of progesterone on glucose: ', round(progesterone_production,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on glucose: ', round(max_yield,5), 'mmol progesterone/mmol glucose') # calculate yield on galactose
    print('')

    medium[Glucose_exchange.id] = 1000 # Set D-glucose to max
    model.medium = medium 

    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution = model.optimize() # simulate model
    progesterone_production = solution.objective_value
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose concentration: ',format(medium[Glucose_exchange.id]))
    print('Maximum theoretical biomass productivity: ', round(solution.fluxes['BIOMASS_SC5_notrace'],5), '/h')
    print('Maximum theoretical productivity of progesterone on glucose: ', round(progesterone_production,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on glucose: ', round(max_yield,5), 'mmol progesterone/mmol glucose') # calculate yield on galactose

Glucose concentration:  10.0
Maximum theoretical biomass productivity:  0.0 /h
Maximum theoretical productivity of progesterone on glucose:  0.14286 mmol/gDW*h
Maximum theoretical yield of progesterone on glucose:  0.01429 mmol progesterone/mmol glucose

Glucose concentration:  500
Maximum theoretical biomass productivity:  0.0 /h
Maximum theoretical productivity of progesterone on glucose:  0.14286 mmol/gDW*h
Maximum theoretical yield of progesterone on glucose:  0.00029 mmol progesterone/mmol glucose

Glucose concentration:  1000
Maximum theoretical biomass productivity:  0.0 /h
Maximum theoretical productivity of progesterone on glucose:  0.14286 mmol/gDW*h
Maximum theoretical yield of progesterone on glucose:  0.00014 mmol progesterone/mmol glucose


Interestingly, it appears as if the productivity of progesterone does not change when the amount of glucose is increased. Instead, the productivity is constant, thus, the yield is lowered as the glucose concentration increases and the productivity remains steady.  Therefore, only increasing the concentration of glucose is not a feasible idea for increasing progesterone yield - something else must be changed to increase the productivity of progesterone, first then, increasing the glucose concentration might be beneficial. As seen before, the biomass productivity is still zero.

## Maximum theoretical productivity and yield of progesterone on alternative carbon sources

To 

In [73]:
with model: #galactose
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Galactose_exchange = model.reactions.get_by_id('EX_gal_e') #define galactose reaction
    medium = model.medium
    medium[Glucose_exchange.id] = 0 # Set glucose to max
    model.medium = medium
    medium[Galactose_exchange.id] = 10 
    model.medium = medium 
    model.objective = model.reactions.R02216 # objective is to lead flux through the reaction producing progesterone top maximise production
    progesterone_production_gal = model.optimize().objective_value # define new model with maximum production of progesterone
    galactose_flux = model.reactions.EX_gal_e.flux
    print(galactose_flux)
    max_yield_gal = progesterone_production_gal/(-1 * galactose_flux)
    print('Galactose concentration: ',format(medium[Galactose_exchange.id]))
    print('Maximum theoretical productivity of progesterone on galactose: ', round(progesterone_production_gal,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on galactose: ', round(max_yield_gal,5), 'mmol progesterone/mmol galactose') # calculate yield on galactose

-6.095238095238098
Galactose concentration:  10
Maximum theoretical productivity of progesterone on galactose:  0.14286 mmol/gDW*h
Maximum theoretical yield of progesterone on galactose:  0.02344 mmol progesterone/mmol galactose


The yield of progesterone is increased when galactose is used as carbon source over glucose.

In [12]:
with model:
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Fructose_exchange = model.reactions.get_by_id('EX_fru_e') #define fructose reaction
    medium = model.medium
    medium[Glucose_exchange.id] = 0 # Set D-glucose to max
    model.medium = medium
    medium[Fructose_exchange.id] = 10 
    model.medium = medium 
    model.objective = model.reactions.R02216 # objective is to lead flux through the reaction producing progesterone top maximise production
    progesterone_production_fru = model.optimize().objective_value # define new model with maximum production of progesterone
    fructose_flux = model.reactions.EX_fru_e.flux
    max_yield_fru = progesterone_production_fru/(-1 * fructose_flux)
    print('Fructose concentration:', format(medium[Fructose_exchange.id]))
    print('Maximum theoretical productivity of progesterone on fructose: ', round(progesterone_production_fru,5), 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on fructose: ', round(max_yield_fru,5), 'mmol progesterone/mmol fructose') # calculate yield on fructose

Fructose concentration: 10
Maximum theoretical productivity of progesterone on fructose:  0.14286 mmol/gDW*h
Maximum theoretical yield of progesterone on fructose:  0.02344 mmol progesterone/mmol fructose


The yield of progesterone is increased when fructose is used as carbon source over glucose, but not as big an increase as galactose. It can be concluded, that if producing progesterone is the main objective, then it could be beneficial to use galactose as carbon source over the default media containing glucose.